In [ ]:
!pip install -U transformers[torch]==4.57.3

In [ ]:
import kagglehub
path = kagglehub.dataset_download("prithvijaunjale/instagram-images-with-captions")

Using Colab cache for faster access to the 'instagram-images-with-captions' dataset.


In [ ]:
import os

print(os.listdir(path))


['instagram_data', 'instagram_data2']


In [ ]:
import pandas as pd

captions_df = pd.read_csv(os.path.join(path, 'instagram_data', 'captions_csv.csv'))
print(captions_df.head())

   Sr No  Image File                                            Caption
0      1  img/insta1                                                NaN
1      2  img/insta2                                               bye 
2      3  img/insta3  Ok, a few more... sorry I just had so much fun...
3      4  img/insta4  This was one of my favorite shoots I’ve ever d...
4      5  img/insta5               Wrapped round my finger like a ring 


In [ ]:
print(captions_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20515 entries, 0 to 20514
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Sr No       20515 non-null  int64 
 1   Image File  20515 non-null  object
 2   Caption     17100 non-null  object
dtypes: int64(1), object(2)
memory usage: 480.9+ KB
None


In [ ]:
captions_df.dropna(subset=['Caption'], inplace=True)
print(captions_df.head())
print(captions_df.info())

   Sr No  Image File                                            Caption
1      2  img/insta2                                               bye 
2      3  img/insta3  Ok, a few more... sorry I just had so much fun...
3      4  img/insta4  This was one of my favorite shoots I’ve ever d...
4      5  img/insta5               Wrapped round my finger like a ring 
5      6  img/insta6  Yesterday was so much fun!! has been such a hu...
<class 'pandas.core.frame.DataFrame'>
Index: 17100 entries, 1 to 20514
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Sr No       17100 non-null  int64 
 1   Image File  17100 non-null  object
 2   Caption     17100 non-null  object
dtypes: int64(1), object(2)
memory usage: 534.4+ KB
None


In [ ]:
captions_df['image_path'] = os.path.join(path, 'instagram_data') + '/' + captions_df['Image File']
print(captions_df.head())

   Sr No  Image File                                            Caption  \
1      2  img/insta2                                               bye    
2      3  img/insta3  Ok, a few more... sorry I just had so much fun...   
3      4  img/insta4  This was one of my favorite shoots I’ve ever d...   
4      5  img/insta5               Wrapped round my finger like a ring    
5      6  img/insta6  Yesterday was so much fun!! has been such a hu...   

                                          image_path  
1  /kaggle/input/instagram-images-with-captions/i...  
2  /kaggle/input/instagram-images-with-captions/i...  
3  /kaggle/input/instagram-images-with-captions/i...  
4  /kaggle/input/instagram-images-with-captions/i...  
5  /kaggle/input/instagram-images-with-captions/i...  


In [ ]:
import torch
from torch.utils.data import Dataset
from transformers import BlipProcessor, AutoModelForVision2Seq
from PIL import Image

In [ ]:
class InstagramCaptioningDataset(Dataset):
    def __init__(self, df, processor):
        self.df = df
        self.processor = processor

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_path = row['image_path']
        caption = row['Caption']

        image = Image.open(image_path).convert("RGB")

        encoding = self.processor(images=image, text=caption, return_tensors="pt")

        # Remove batch dimension
        encoding = {k: v.squeeze() for k, v in encoding.items()}

        return encoding

In [ ]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
print("BLIP processor instantiated.")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

BLIP processor instantiated.


In [ ]:
from transformers import BlipForConditionalGeneration

model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
print("BLIP model loaded.")

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

BLIP model loaded.


In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

print("Imported Seq2SeqTrainingArguments and Seq2SeqTrainer.")

Imported Seq2SeqTrainingArguments and Seq2SeqTrainer.


In [ ]:
import re
import csv
from pathlib import Path


def remove_emojis(text):
    """Remove emojis from text"""
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F1E0-\U0001F1FF"  # flags (iOS)
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "\U0001F900-\U0001F9FF"  # supplemental symbols
        "\U0001FA00-\U0001FAFF"  # extended symbols
        "]+",
        flags=re.UNICODE
    )
    return emoji_pattern.sub(r'', text)


def remove_mentions(text):
    """Remove @mentions from text"""
    return re.sub(r'@\w+', '', text)


def remove_urls(text):
    """Remove URLs from text"""
    url_pattern = re.compile(
        r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    )
    text = url_pattern.sub('', text)
    # Also remove www. URLs
    text = re.sub(r'www\.[^\s]+', '', text)
    return text


def remove_hashtags(text):
    """Remove #hashtags from text"""
    return re.sub(r'#\w+', '', text)


def clean_caption(caption):
    """Apply all cleaning operations to a caption"""
    if not caption:
        return ""
    if not isinstance(caption, str):
        return ""
    # Apply all cleaning steps
    caption = remove_emojis(caption)
    caption = remove_mentions(caption)
    caption = remove_urls(caption)
    caption = remove_hashtags(caption)

    # Clean up extra whitespace
    caption = re.sub(r'\s+', ' ', caption)
    caption = caption.strip()

    return caption

In [ ]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split

# Re-read the CSV as captions_df was emptied
captions_df = pd.read_csv(os.path.join(path, 'instagram_data', 'captions_csv.csv'))

# Apply the cleaning function to the 'Caption' column
captions_df['Caption'] = captions_df['Caption'].apply(clean_caption)

# Drop rows with null captions (as done previously)
captions_df.dropna(subset=['Caption'], inplace=True)

#drop rows with blank caption
captions_df = captions_df[captions_df['Caption'] != '']


# Correctly construct image_path by appending '.jpg' extension
# and using os.path.join for robust path construction
captions_df['image_path'] = captions_df['Image File'].apply(lambda x: os.path.join(path, 'instagram_data', x + '.jpg'))

# Filter out rows where the image file does not exist
initial_rows = len(captions_df)
captions_df = captions_df[captions_df['image_path'].apply(os.path.exists)]
filtered_rows = len(captions_df)

print(f"Re-loaded and filtered captions_df. Removed {initial_rows - filtered_rows} rows due to missing image files or incorrect extensions.")
print(f"Remaining rows: {filtered_rows}")
print(captions_df.head())

# Re-split the data since captions_df has changed
train_df, val_df = train_test_split(captions_df, test_size=0.2, random_state=42)

print(f"Training set size: {len(train_df)}")
print(f"Validation set size: {len(val_df)}")

Re-loaded and filtered captions_df. Removed 0 rows due to missing image files or incorrect extensions.
Remaining rows: 15756
   Sr No  Image File                                            Caption  \
1      2  img/insta2                                                bye   
2      3  img/insta3  Ok, a few more... sorry I just had so much fun...   
3      4  img/insta4  This was one of my favorite shoots I’ve ever d...   
4      5  img/insta5                Wrapped round my finger like a ring   
5      6  img/insta6  Yesterday was so much fun!! has been such a hu...   

                                          image_path  
1  /kaggle/input/instagram-images-with-captions/i...  
2  /kaggle/input/instagram-images-with-captions/i...  
3  /kaggle/input/instagram-images-with-captions/i...  
4  /kaggle/input/instagram-images-with-captions/i...  
5  /kaggle/input/instagram-images-with-captions/i...  
Training set size: 12604
Validation set size: 3152


In [ ]:
import torch

def data_collator(features):
    # Extract pixel_values, input_ids, attention_mask, and labels
    pixel_values = torch.stack([feature["pixel_values"] for feature in features])
    input_ids = [feature["input_ids"] for feature in features]
    attention_mask = [feature["attention_mask"] for feature in features]
    labels = [feature["labels"] for feature in features]

    # Pad the input_ids and attention_mask using the processor's tokenizer
    # The tokenizer.pad method expects a dictionary of lists for batch padding
    padded_inputs = processor.tokenizer.pad(
        {'input_ids': input_ids, 'attention_mask': attention_mask},
        return_tensors="pt",
        padding=True
    )

    padded_input_ids = padded_inputs['input_ids']
    padded_attention_mask = padded_inputs['attention_mask']

    # For labels, we only need to pad `input_ids`
    # We need to explicitly pass it as a dictionary with 'input_ids' key for padding
    padded_labels_dict = processor.tokenizer.pad(
        {'input_ids': labels},
        return_tensors="pt",
        padding=True
    )
    padded_labels = padded_labels_dict['input_ids']

    # Replace padding token id in labels with -100 (which is ignored by PyTorch's CrossEntropyLoss)
    padded_labels[padded_labels == processor.tokenizer.pad_token_id] = -100

    batch = {
        "pixel_values": pixel_values,
        "input_ids": padded_input_ids,
        "attention_mask": padded_attention_mask,
        "labels": padded_labels,
    }
    return batch

print("Data collator function defined with corrected padding.")

Data collator function defined with corrected padding.


In [ ]:
from transformers import Seq2SeqTrainer

class CustomSeq2SeqTrainer(Seq2SeqTrainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        # num_items_in_batch is an argument passed by the Trainer's training_step method,
        # but not expected by the base compute_loss or the model's forward method.
        # By accepting it here, we prevent the TypeError.
        # We do not pass it to super().compute_loss as it's not part of its signature.
        return super().compute_loss(model, inputs, return_outputs=return_outputs)

print("CustomSeq2SeqTrainer defined to handle 'num_items_in_batch' argument.")

CustomSeq2SeqTrainer defined to handle 'num_items_in_batch' argument.


In [ ]:
class InstagramCaptioningDataset(Dataset):
    def __init__(self, df, processor):
        self.df = df
        self.processor = processor

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_path = row['image_path']
        caption = row['Caption']

        image = Image.open(image_path).convert("RGB")

        # Add max_length and truncation to handle varying caption lengths
        encoding = self.processor(images=image, text=caption, return_tensors="pt",
                                  max_length=512, truncation=True)

        # Remove batch dimension
        encoding = {k: v.squeeze() for k, v in encoding.items()}

        # Add 'labels' key for sequence-to-sequence training
        encoding["labels"] = encoding["input_ids"]

        return encoding
print("InstagramCaptioningDataset class redefined with max_length and truncation for processor.")

InstagramCaptioningDataset class redefined with max_length and truncation for processor.


In [ ]:
train_dataset = InstagramCaptioningDataset(train_df, processor)
val_dataset = InstagramCaptioningDataset(val_df, processor)

print(f"Training dataset recreated with {len(train_dataset)} samples.")
print(f"Validation dataset recreated with {len(val_dataset)} samples.")

Training dataset recreated with 12604 samples.
Validation dataset recreated with 3152 samples.


In [ ]:
from transformers import IntervalStrategy # Added import for IntervalStrategy

training_args = Seq2SeqTrainingArguments(
    output_dir='/content/drive/MyDrive/blip_fine_tuned_checkpoints',
    num_train_epochs=5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=5e-5,
    weight_decay=0.01,
    save_total_limit=1,
    predict_with_generate=True,
    # evaluation_strategy=IntervalStrategy.EPOCH, # Removed due to TypeError
    logging_dir='/content/drive/MyDrive/blip_fine_tuned_checkpoints/logs',
    remove_unused_columns=False,
    report_to='none',
    gradient_checkpointing=True # Enabled gradient checkpointing
)

print("Training arguments defined (batch size reduced and gradient checkpointing enabled). Output directory set to Google Drive).")

Training arguments defined (batch size reduced and gradient checkpointing enabled). Output directory set to Google Drive).


In [ ]:
trainer = CustomSeq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

print("Seq2SeqTrainer instance re-created with updated training arguments (gradient checkpointing enabled and output directory set to Google Drive).")

Seq2SeqTrainer instance re-created with updated training arguments (gradient checkpointing enabled and output directory set to Google Drive).


In [ ]:
trainer.train(resume_from_checkpoint='/content/drive/MyDrive/blip_fine_tuned_checkpoints/checkpoint-9500')

print("Training process initiated, resuming from checkpoint.")

There were missing keys in the checkpoint model loaded: ['text_decoder.cls.predictions.decoder.weight', 'text_decoder.cls.predictions.decoder.bias'].


Step,Training Loss
10000,1.961500
10500,1.934000
11000,1.925100
11500,1.937600
12000,1.927100
12500,1.914500
13000,1.453500
13500,1.266600
14000,1.281700
14500,1.310200


Training process initiated, resuming from checkpoint.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
model_save_path = '/content/drive/MyDrive/blip_fine_tuned_model'

# Save the fine-tuned model
trainer.save_model(model_save_path)

# Save the processor as well
processor.save_pretrained(model_save_path)

print(f"Fine-tuned model and processor saved to: {model_save_path}")

Fine-tuned model and processor saved to: /content/drive/MyDrive/blip_fine_tuned_model


In [ ]:
import matplotlib.pyplot as plt

# Select a small sample of unseen images from val_df
sample_images_df = val_df.sample(n=10)
images = []

original_captions = []
preprocessed_images = []

for index, row in sample_images_df.iterrows():
    image_path = row['image_path']
    original_caption = row['Caption']

    # Load the image
    image = Image.open(image_path).convert("RGB")
    images.append(image)
    original_captions.append(original_caption)

    # Preprocess the image
    pixel_values = processor(images=image, return_tensors="pt").pixel_values
    preprocessed_images.append(pixel_values)

print(f"Selected {len(images)} sample images for caption generation.")
print([img.shape for img in preprocessed_images])

Selected 12 sample images for caption generation.
[torch.Size([1, 3, 384, 384]), torch.Size([1, 3, 384, 384]), torch.Size([1, 3, 384, 384]), torch.Size([1, 3, 384, 384]), torch.Size([1, 3, 384, 384]), torch.Size([1, 3, 384, 384]), torch.Size([1, 3, 384, 384]), torch.Size([1, 3, 384, 384]), torch.Size([1, 3, 384, 384]), torch.Size([1, 3, 384, 384])]


In [ ]:
from IPython.display import display
import glob

# --- USER INPUT REQUIRED ---
# Specify the path to your custom images directory in Google Drive here.
custom_images_dir = '/content/drive/MyDrive/images' # Example: '/content/drive/MyDrive/my_images'

custom_image_paths = []
if custom_images_dir:
    # Use glob to find all common image files in the directory
    for ext in ['*.jpg', '*.jpeg', '*.png']:
        custom_image_paths.extend(glob.glob(f"{custom_images_dir}/{ext}"))

if not custom_image_paths:
    print(f"No images found in '{custom_images_dir}' or directory does not exist. Please check the path and image files.")
else:
    custom_images = []
    custom_preprocessed_images = []

    for img_path in custom_image_paths:
        try:
            image = Image.open(img_path).convert("RGB")
            custom_images.append(image)
            pixel_values = processor(images=image, return_tensors="pt").pixel_values
            custom_preprocessed_images.append(pixel_values)
        except FileNotFoundError:
            print(f"Warning: Image not found at {img_path}. Skipping this image.")
        except Exception as e:
            print(f"Error processing image {img_path}: {e}. Skipping this image.")

    if custom_images:
        custom_generated_captions = []

        model.eval() # Ensure model is in evaluation mode
        if hasattr(model, "gradient_checkpointing_disable"):
            model.gradient_checkpointing_disable()

        for pixel_values in custom_preprocessed_images:
            with torch.no_grad():
                outputs = model.generate(
                    pixel_values.to(model.device),
                    max_new_tokens=50,
                    min_length=1,
                    num_beams=1,
                    do_sample=False,
                    early_stopping=True
                )
            decoded_caption = processor.tokenizer.batch_decode(outputs.cpu(), skip_special_tokens=True)[0]
            custom_generated_captions.append(decoded_caption)

        # Display each custom image with its generated caption
        print("\n--- Generated Captions for Custom Images ---")
        for i, image in enumerate(custom_images):
            display(image)
            print(f"Generated Caption: {custom_generated_captions[i]}\n")

    print("Finished generating captions for custom images.")

In [ ]:
generated_captions = []

# It's good practice to set the model to evaluation mode for inference
model.eval()

# Disable gradient checkpointing if it's enabled (for inference)
# The BlipForConditionalGeneration model has a gradient_checkpointing_disable() method
if hasattr(model, "gradient_checkpointing_disable"):
    model.gradient_checkpointing_disable()

for pixel_values in preprocessed_images:
    with torch.no_grad(): # Disable gradient computation for efficiency
        outputs = model.generate(
            pixel_values.to(model.device),
            max_new_tokens=50,
            min_length=1,
            num_beams=1,
            do_sample=False,
            early_stopping=True
        )

    decoded_caption = processor.tokenizer.batch_decode(outputs.cpu(), skip_special_tokens=True)[0]
    generated_captions.append(decoded_caption)

# If gradient checkpointing was originally enabled, and needs to be re-enabled for subsequent tasks,
# it should be done here using `model.gradient_checkpointing_enable()`. However, for pure inference,
# it's often left disabled. Since we couldn't reliably store its original state (due to the AttributeError),
# we will assume it's not necessary to re-enable it immediately after generation.

print("Generated captions for sample images.")

Generated captions for sample images.


In [ ]:
import matplotlib.pyplot as plt

for i, image in enumerate(images):
    plt.figure(figsize=(8, 8))
    plt.imshow(image)
    plt.axis('off')
    plt.title(f"Original Caption: {original_captions[i]}\nGenerated Caption: {generated_captions[i]}")
    plt.show()

print("Visualized original images with generated captions.")

In [ ]:
from IPython.display import display

# Select a new small sample of unseen images from val_df
sample_images_df_new = val_df.sample(n=3)

new_images = []
new_original_captions = []
new_preprocessed_images = []

for index, row in sample_images_df_new.iterrows():
    image_path = row['image_path']
    original_caption = row['Caption']

    # Load the image
    image = Image.open(image_path).convert("RGB")
    new_images.append(image)
    new_original_captions.append(original_caption)

    # Preprocess the image
    pixel_values = processor(images=image, return_tensors="pt").pixel_values
    new_preprocessed_images.append(pixel_values)

print(f"Selected {len(new_images)} new sample images for caption generation.")

new_generated_captions = []

for pixel_values in new_preprocessed_images:
    # Generate captions
    outputs = model.generate(pixel_values.to(model.device))

    # Move to CPU and decode
    decoded_caption = processor.tokenizer.batch_decode(outputs.cpu(), skip_special_tokens=True)[0]
    new_generated_captions.append(decoded_caption)

print("Generated captions for new sample images.")

# Display each image with its original and generated captions
for i, image in enumerate(new_images):
    display(image)
    print(f"Original Caption: {new_original_captions[i]}")
    print(f"Generated Caption: {new_generated_captions[i]}\n")

print("Visualized new original images with generated captions.")

Selected 3 new sample images for caption generation.


/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


RuntimeError: The size of tensor a (579) must match the size of tensor b (2) at non-singleton dimension 3